In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from Training import GetDatasets
from Nets  import GetNeuralNetworkModel
from Stats import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from Simulation import buffer_simulation, save_obj, load_obj


tf.random.set_seed(1234)
np.random.seed(1234)

a) Get Dataset

In [3]:
#train_batch_size = test_batch_size = 32
#train_set,valid_set,test_set = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, train_batch_size, test_batch_size)
train_batch_size = test_batch_size = 32
train_set,valid_set,test_set = GetDatasets('colorectal_histology',(80,5,15),(224,224), 8, train_batch_size, test_batch_size)

In [4]:
train_set

<RepeatDataset shapes: ((None, 224, 224, 3), (None, 8)), types: (tf.float32, tf.float32)>

In [8]:
SqueezeNet     = GetNeuralNetworkModel('SqueezeNet',(224,224,3),8, quantization = False, aging_active=False)
loss      = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
SqueezeNet.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


In [6]:
SqueezeNet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 224, 224, 3)  0           lambda[0][0]                     
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 96) 14208       lambda_1[0][0]                   
______________________________________________________________________________________________

a) Get Model

Se crea la red, sin activar la cuantización ni el efecto de envejecimiento
Las dimensiones de entrada de la imagen (224,224), el número de clases (8) y el tamaño de los batches

 b) Load/Save Weigths

In [10]:
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'SqueezeNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir, 'Weights') 
SqueezeNet.load_weights(wgt_dir)

Evaluación del accuracy y loss de la red

In [11]:
(OrigLoss,OrigAcc) = SqueezeNet.evaluate(test_set)


24/24 [==============================] - 15s 638ms/step - loss: 0.2379 - accuracy: 0.9320


2) Stats

 Write/Read Stats¶

Se identifican (manualmente) las capas procesadadas(Convoluciones, Full conectadas y Pooling) junto a las capas que contienen los resultados que se escribiran en el buffer (capas luego de la funcion de activacion y/o Normalizacion)

In [12]:
for index,layer in enumerate(SqueezeNet.layers):
    print(index,layer.name)
print('Las capas 0,3,9,11,17,19,25,31,37,40,45 y 50  contienen la informacion para su procesamiento en MMU')
print('Las capas 2,8,10,16,18,24,30,36,38,44,49 y 53 contienen las activaciones que son escritas en memoria')

0 input_2
1 lambda_43
2 lambda_44
3 conv2d_26
4 re_lu_25
5 lambda_45
6 lambda_46
7 max_pooling2d_3
8 lambda_47
9 conv2d_27
10 re_lu_26
11 lambda_48
12 lambda_49
13 conv2d_28
14 conv2d_29
15 re_lu_27
16 re_lu_28
17 concatenate_8
18 lambda_50
19 lambda_51
20 conv2d_30
21 re_lu_29
22 lambda_52
23 lambda_53
24 conv2d_31
25 conv2d_32
26 re_lu_30
27 re_lu_31
28 concatenate_9
29 lambda_54
30 lambda_55
31 conv2d_33
32 re_lu_32
33 lambda_56
34 lambda_57
35 conv2d_34
36 conv2d_35
37 re_lu_33
38 re_lu_34
39 concatenate_10
40 lambda_58
41 lambda_59
42 max_pooling2d_4
43 lambda_60
44 conv2d_36
45 re_lu_35
46 lambda_61
47 lambda_62
48 conv2d_37
49 conv2d_38
50 re_lu_36
51 re_lu_37
52 concatenate_11
53 lambda_63
54 lambda_64
55 conv2d_39
56 re_lu_38
57 lambda_65
58 lambda_66
59 conv2d_40
60 conv2d_41
61 re_lu_39
62 re_lu_40
63 concatenate_12
64 lambda_67
65 lambda_68
66 conv2d_42
67 re_lu_41
68 lambda_69
69 lambda_70
70 conv2d_43
71 conv2d_44
72 re_lu_42
73 re_lu_43
74 concatenate_13
75 lambda_71
76 

Con el siguiente bloque obtenemos el número de lecturas y escrituras por posición de memoria tanto usando la estrategia de CNN Gating o sin usarla

In [13]:
num_address= 1024*1024
samples      = 10
Indices = [0,3,7, 9,(13,14),20,(24,25),31,(35,36),42,44,(48,49),55,(59,60),66,(70,71),77,(81,82),88,90,(94,95),101,104]
Data    = GetReadAndWrites(SqueezeNet,Indices,801816,150,CNN_gating=False,network_name='SqueezeNet')
stats   = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
Baseline_Acceses   = pd.DataFrame(stats).reset_index(drop=False)
Data    = GetReadAndWrites(SqueezeNet,Indices,num_address,samples,CNN_gating=True,network_name='SqueezeNet')
stats    = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
CNN_gating_Acceses = pd.DataFrame(stats).reset_index(drop=False)
save_obj(Baseline_Acceses,'Data/Acceses/SqueezeNet/Baseline_v2')
save_obj(CNN_gating_Acceses,'Data/Acceses/SqueezeNet/CNN_gating_Fix_v2')

procesados:  25
procesados:  50
procesados:  75
procesados:  100
procesados:  125
procesados:  150


# a) Number of bits analysis

 Ahora veremos como afecta la cuantizacion, tanto en pesos como en activaciones a la accuracy y loss

In [ ]:
df = QuantizationEffect('SqueezeNet',test_set,wgt_dir,(224,224,3),8,test_batch_size)
#save_obj(df,'Data/Quantization/AlexNet/Colorectal Dataset/Quantization')

Activation fraction part
0  bits results:  acc:  0.2213333398103714 loss:  nan


C:\Users\usuario\Desktop\CNN_Gating\Stats.py:227: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Experiment':['Activation fraction part'],'bits':[bit],'acc':[acc],'loss':[loss]}))


1  bits results:  acc:  0.4346666634082794 loss:  8.266940116882324
2  bits results:  acc:  0.6666666865348816 loss:  3.216773509979248
3  bits results:  acc:  0.8320000171661377 loss:  1.4068776369094849


C:\Users\usuario\Desktop\CNN_Gating\Stats.py:227: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Experiment':['Activation fraction part'],'bits':[bit],'acc':[acc],'loss':[loss]}))


4  bits results:  acc:  0.9306666851043701 loss:  0.4856926202774048


...

C:\Users\usuario\Desktop\CNN_Gating\Stats.py:239: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Experiment':['Activation integer part'],'bits':[bit],'acc':[acc],'loss':[loss]}))


5  bits results:  acc:  0.9240000247955322 loss:  0.23931172490119934


C:\Users\usuario\Desktop\CNN_Gating\Stats.py:239: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Experiment':['Activation integer part'],'bits':[bit],'acc':[acc],'loss':[loss]}))


6  bits results:  acc:  0.9319999814033508 loss:  0.24338509142398834


C:\Users\usuario\Desktop\CNN_Gating\Stats.py:239: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Experiment':['Activation integer part'],'bits':[bit],'acc':[acc],'loss':[loss]}))


In [16]:
CheckAccuracyAndLoss('SqueezeNet', test_set, wgt_dir, act_frac_size = 9, act_int_size = 6, wgt_frac_size = 15, wgt_int_size = 0, 
                    input_shape = (224,224,3), output_shape = 8, batch_size = test_batch_size)

24/24 [==============================] - 24s 987ms/step - loss: 0.4706 - accuracy: 0.8080


 c) Activation Stats

Para la configuración anterior, se observa el valor medio,maximo,minimo y el ratio de saturación tanto de las activaciones procesadas dentro de la unidad matricial de multiplicacion como de las almacenadas en el buffer. Nota: el ultimo parametro indica el numero de iteraciones que se deben realizar hasta agotar el dataset, se obtiene como numero de imagenes del dataset dividido en el batch size.

In [10]:
ActivationStats(SqueezeNet,test_set,9,6,24)

mean value (MMU): -0.16231656
mean value (Buffer): 0.14423333
maximum (MMU): 106.956314
minimum (MMU): -188.0446
maximum (Buffer): 106.956314
minimum (Buffer): -131.6606
saturation ratio (MMU): 0.0022129352593187345
saturation ratio (Buffer): 0.00017893128801773947


3) Buffer Simulation

 a) Baseline

Ahora para el Baseline simularemos el comportamiento de 1 buffer durante la inferencia de 3 imagenes (solo 3 como ejemplo), la red se crea ahora activando la cuantizacion pero no el envejecimiento. LI y AI son los definidos en el item 2) Stats

......estos ficheros ya están , solo cargar e interpretarlos

Sin inyectar errores original

In [12]:
# from copy import deepcopy
from Stats import CheckAccuracyAndLoss
from Simulation import save_obj, load_obj
from datetime import datetime
import itertools
from Training import GetDatasets
import numpy as np
import os

cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'SqueezeNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')
    

trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)


Accs_x= []



network_size   = 290400*16   # Tamaño del buffer (en bits)
num_of_samples = 1       # Numero de muestras de distintas configuraciones de fallos a testear por cada valor de Accs/Loss
    
n_bits_total = np.ceil(network_size).astype(int)    #numero de bits totales
    
buffer = np.array(['x']*(network_size))
print(buffer)
for index in range(0,num_of_samples):
        
    loss,acc_x   = CheckAccuracyAndLoss('SqueezeNet', test_dataset, wgt_dir, output_shape=8, input_shape = (224,224,3),
                                            act_frac_size = 9, act_int_size = 6, wgt_frac_size = 15, wgt_int_size = 0,
                                            batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = False)
       
        
    print(acc_x)
        
      
        
    Accs_x.append(acc_x)
        
print(str(n_bits_total)+' completada: ', datetime.now().strftime("%H:%M:%S"))
#save_obj(Accs_x,'Data/Errors/SqueezeNet/Colorectal Dataset/Accs_x')
#save_obj(Loss,'Data/Errors/SqueezeNet/Colorectal Dataset/Loss')

['x' 'x' 'x' ... 'x' 'x' 'x']
0.8080000281333923
4646400 completada:  09:01:54


# Analizando el fichero en su estado original

In [1]:
## from copy import deepcopy
from Stats import CheckAccuracyAndLoss
from Simulation import save_obj, load_obj
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
import collections
from datetime import datetime
import itertools
from Training import GetDatasets
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import os, sys

 
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'SqueezeNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')
    

trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(224,224), 8, trainBatchSize, testBatchSize)


Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216

       
ruta_bin = 'Data\Fault Characterization\VC707\RawData'
#ruta_bin = 'Data/Fault Characterization/KC705_B/RowData/'
directorio = pathlib.Path(ruta_bin)
num_of_samples = 1

variantes= [analize_file, analize_file_uno, analize_file_uno_ceros]
ficheros = [fichero.name for fichero in directorio.iterdir()]
for index in range(0,num_of_samples):
    for i, j in enumerate(ficheros):
        directorio= os.path.join(ruta_bin, j)
        buffer= (analize_file(directorio, buffer_size))
        error_mask, locs = (buffer_vectores(buffer))
    
        loss,acc   = CheckAccuracyAndLoss('SqueezeNet', test_dataset, wgt_dir, output_shape=8, input_shape = (224,224,3),
                                            act_frac_size = 9, act_int_size = 5, wgt_frac_size = 15, wgt_int_size = 0,
                                            batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
        loss,acc_w   = CheckAccuracyAndLoss('SqueezeNet', test_dataset, wgt_dir, output_shape=8, input_shape = (224,224,3),
                                            act_frac_size = 9, act_int_size = 5, wgt_frac_size = 15, wgt_int_size = 0,
                                            batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
        loss,acc_a_w   = CheckAccuracyAndLoss('SqueezeNet', test_dataset, wgt_dir, output_shape=8, input_shape = (224,224,3),
                                            act_frac_size = 9, act_int_size = 5, wgt_frac_size = 15, wgt_int_size = 0,
                                            batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    
        Accs.append(acc)
        Accs_w.append(acc_w)
        Accs_a_w.append(acc_a_w)                                      


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_w)                                         
#buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
#buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='ceros_707', index=False)


Acc.head()
    
   



    
        
    

print(str()+' operación completada: ', datetime.now().strftime("%H:%M:%S"))
#lsave_file(Accs,'Data/Fault Characterization/Accs')

#save_obj(Loss,'Data/Errors/SqueezeNet/Colorectal Dataset/Loss')

Cantidad de errores de todo el fihero: 23706
Cantidad de elementos por tipo : Counter({'x': 16766856, '0': 10360})
16777216
16777216


InvalidArgumentError:  indices[8447] = [15, 199, 142, 2] does not index into param shape [14,227,227,3], node name: model/lambda_1/GatherNd
	 [[node model/lambda_1/GatherNd (defined at C:\Users\usuario\Desktop\CNN_Gating\Nets.py:79) ]] [Op:__inference_test_function_5368]

Errors may have originated from an input operation.
Input Source operations connected to node model/lambda_1/GatherNd:
 model/lambda/Maximum (defined at C:\Users\usuario\Desktop\CNN_Gating\Nets.py:54)

Function call stack:
test_function


# Inyectando errores en 1

In [2]:
from Stats import CheckAccuracyAndLoss
from Simulation import save_obj, load_obj
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
import collections
from datetime import datetime
import itertools
from Training import GetDatasets
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import os, sys

trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)




    

cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'SqueezeNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')
    

Accs_1 = []
Accs_w_1 = []
Accs_a_w_1 = []



df = pd.DataFrame(columns=['Accs', 'Acc_w'])
buffer_size= 16777216
num_of_samples = 1


       
ruta_bin = 'Data\Fault Characterization\VC707\RawData'
#ruta_bin = 'Data/Fault Characterization/KC705_B/RowData/'
directorio = pathlib.Path(ruta_bin)

variantes= [analize_file, analize_file_uno, analize_file_uno_ceros]
ficheros = [fichero.name for fichero in directorio.iterdir()]
print(directorio)
for index in range(0,num_of_samples):
    for i, j in enumerate(ficheros):
        directorio= os.path.join(ruta_bin, j)
        buffer= (analize_file_uno(directorio, buffer_size))
        error_mask, locs = (buffer_vectores(buffer))
        print(directorio)
        
        loss,acc_1   = CheckAccuracyAndLoss('SqueezeNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = False,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        loss,acc_w_1   = CheckAccuracyAndLoss('SqueezeNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = True,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        loss,acc_a_w_1   = CheckAccuracyAndLoss('SqueezeNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = True,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        
        Accs_1.append(acc_1)
        Accs_w_1.append(acc_w_1)
        Accs_a_w_1.append(acc_a_w_1)                                      


Acc_1=pd.DataFrame(Accs_1)
Acc_w_1 =pd.DataFrame(Accs_w_1)
Acc_a_w_1 =pd.DataFrame(Accs_w_1)                                         
#buf_cero = pd.concat([Acc_1,Acc_w_1, Acc_a_w_1], axis=1, join='outer')
#buf_cero.columns =['Acc_uno', 'A_w_uno', 'Acc_a_w_uno'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='unos_707', index=False)

#buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
#buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='ceros_707', index=False)



Acc.head()
    
   



    
        
    

print(str()+' operación completada: ', datetime.now().strftime("%H:%M:%S"))
save_file(Accs_1,'Data/Fault Characterization/Accs')

#save_obj(Loss,'Data/Errors/AlexNet/Colorectal Dataset/Loss')

Data\Fault Characterization\VC707\RawData
Cantidad de errores de todo el fihero: 23706
Cantidad de elementos por tipo : Counter({'x': 16766856, '1': 10360})
16777216
Data\Fault Characterization\VC707\RawData\VC707-0.54.bin


InvalidArgumentError:  indices[8447] = [15, 199, 142, 2] does not index into param shape [14,227,227,3], node name: model_1/lambda_44/GatherNd
	 [[node model_1/lambda_44/GatherNd (defined at C:\Users\usuario\Desktop\CNN_Gating\Nets.py:79) ]] [Op:__inference_test_function_10827]

Errors may have originated from an input operation.
Input Source operations connected to node model_1/lambda_44/GatherNd:
 model_1/lambda_43/Maximum (defined at C:\Users\usuario\Desktop\CNN_Gating\Nets.py:54)

Function call stack:
test_function


# Inyectando errores aleatorios ceros y unos

In [3]:
buffer_size= 16777216
Accs_1_0 = []
Accs_w_1_0 = []
Accs_a_w_1_0 = []



df = pd.DataFrame(columns=['Accs', 'Acc_w'])
buffer_size= 16777216
num_of_samples = 1


       
ruta_bin = 'Data\Fault Characterization\VC707\RawData'
#ruta_bin = 'Data/Fault Characterization/KC705_B/RowData/'
directorio = pathlib.Path(ruta_bin)

variantes= [analize_file, analize_file_uno, analize_file_uno_ceros]
ficheros = [fichero.name for fichero in directorio.iterdir()]

for index in range(0,num_of_samples):

    for i, j in enumerate(ficheros):
        directorio= os.path.join(ruta_bin, j)
        buffer= (analize_file_uno_ceros(directorio, buffer_size))
        error_mask, locs = (buffer_vectores(buffer))
        print(directorio)
        
        loss,acc_1_0   = CheckAccuracyAndLoss('SqueezeNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = False,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        loss,acc_w_1_0   = CheckAccuracyAndLoss('SqueezeNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = True,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        loss,acc_a_w_1_0   = CheckAccuracyAndLoss('SqueezeNet', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = True,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        
        Accs_1_0.append(acc_1_0)
        Accs_w_1_0.append(acc_w_1_0)
        Accs_a_w_1_0.append(acc_a_w_1_0)                                      


Acc_1_0=pd.DataFrame(Accs_1_0)
Acc_w_1_0 =pd.DataFrame(Accs_w_1_0)
Acc_a_w_1_0 =pd.DataFrame(Accs_w_1_0)                                         
buf_cero = pd.concat([Acc,Acc_w, Acc_a_w,Acc_1,Acc_w_1, Acc_a_w_1,Acc_1_0,Acc_w_1_0, Acc_a_w_1_0], axis=1, join='outer')
buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero', 'Acc_uno', 'A_w_uno', 'Acc_a_w_uno' ,'Acc_uno_cero', 'A_w_uno_cero', 'Acc_a_w_uno_cero'] 
buf_cero.to_excel('resultado.xlsx', sheet_name='fichero_707', index=False)

#buf_cero = pd.concat([Acc_1,Acc_w_1, Acc_a_w_1], axis=1, join='outer')
#buf_cero.columns =['Acc_uno', 'A_w_uno', 'Acc_a_w_uno'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='unos_707', index=False)

#buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
#buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='ceros_707', index=False)
Acc.head()
    
   



    
        
    

print(str()+' operación completada: ', datetime.now().strftime("%H:%M:%S"))
#save_file(Accs_1,'Data/Fault Characterization/Accs')

#save_obj(Loss,'Data/Errors/SqueezeNet/Colorectal Dataset/Loss')

Cantidad de errores de todo el fihero: 23706
Cantidad de elementos por tipo : Counter({'x': 16766856, '0': 5196, '1': 5164})
16777216
Data\Fault Characterization\VC707\RawData\VC707-0.54.bin


InvalidArgumentError:  indices[8447] = [15, 199, 142, 2] does not index into param shape [14,227,227,3], node name: model_2/lambda_87/GatherNd
	 [[node model_2/lambda_87/GatherNd (defined at C:\Users\usuario\Desktop\CNN_Gating\Nets.py:79) ]] [Op:__inference_test_function_15907]

Errors may have originated from an input operation.
Input Source operations connected to node model_2/lambda_87/GatherNd:
 model_2/lambda_86/Maximum (defined at C:\Users\usuario\Desktop\CNN_Gating\Nets.py:54)

Function call stack:
test_function


# Gráficas